In [20]:
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Activation, Dropout
from tensorflow.keras import applications
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras import Model
from tensorflow.keras.applications import resnet
from tensorflow.keras.callbacks import ModelCheckpoint



target_shape = (224, 224)

In [21]:
def preprocess_image(filename):
    """
    Load the specified file as a JPEG image, preprocess it and
    resize it to the target shape.
    """

    image_string = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, target_shape)
    return image


def preprocess_triplets(anchor, positive, negative):
    """
    Given the filenames corresponding to the three images, load and
    preprocess them.
    """

    return (
        preprocess_image(anchor),
        preprocess_image(positive),
        preprocess_image(negative),
    )

In [22]:
test_triplets = pd.read_csv('/Users/davidschwartz/Documents/GitHub/IML-Projects/Task 3/test_triplets.txt', sep=' ', header=None)
train_triplets = pd.read_csv('/Users/davidschwartz/Documents/GitHub/IML-Projects/Task 3/train_triplets.txt', sep=' ', header=None)

anchor_indexes=train_triplets.iloc[0:10000:,0]
print(len(anchor_indexes))
print(anchor_indexes)
positives_indexes=train_triplets.iloc[0:10000,1]
negatives_indexes=train_triplets.iloc[0:10000,2]

10000
0       2461
1       2299
2       4663
3       4532
4       3454
        ... 
9995     723
9996    2791
9997    1394
9998    2735
9999    3675
Name: 0, Length: 10000, dtype: int64


In [23]:
str_path = '/Users/davidschwartz/Documents/GitHub/IML-Projects/Task 3/food'
anchor_images=list()
for number in anchor_indexes:
    anchor_images.append(str_path+'/'+str(number).rjust(5, '0')+'.jpg')
image_string = tf.io.read_file(anchor_images[0])
print(anchor_images[0:3])
anchor_dataset = tf.data.Dataset.from_tensor_slices(anchor_images)

positives_images=list()
for number in positives_indexes:
    positives_images.append(str_path+'/'+str(number).rjust(5, '0')+'.jpg')
image_string = tf.io.read_file(positives_images[0])
print(positives_images[0:3])
positive_dataset = tf.data.Dataset.from_tensor_slices(positives_images)

negatives_images=list()
for number in negatives_indexes:
    negatives_images.append(str_path+'/'+str(number).rjust(5, '0')+'.jpg')
image_string = tf.io.read_file(negatives_images[0])
print(negatives_images[0:3])
negative_dataset = tf.data.Dataset.from_tensor_slices(negatives_images)

image_count = len(anchor_images)

['/Users/davidschwartz/Documents/GitHub/IML-Projects/Task 3/food/02461.jpg', '/Users/davidschwartz/Documents/GitHub/IML-Projects/Task 3/food/02299.jpg', '/Users/davidschwartz/Documents/GitHub/IML-Projects/Task 3/food/04663.jpg']
['/Users/davidschwartz/Documents/GitHub/IML-Projects/Task 3/food/03450.jpg', '/Users/davidschwartz/Documents/GitHub/IML-Projects/Task 3/food/02499.jpg', '/Users/davidschwartz/Documents/GitHub/IML-Projects/Task 3/food/01056.jpg']
['/Users/davidschwartz/Documents/GitHub/IML-Projects/Task 3/food/02678.jpg', '/Users/davidschwartz/Documents/GitHub/IML-Projects/Task 3/food/04987.jpg', '/Users/davidschwartz/Documents/GitHub/IML-Projects/Task 3/food/03029.jpg']


In [24]:
dataset = tf.data.Dataset.zip((anchor_dataset, positive_dataset, negative_dataset))
dataset = dataset.shuffle(buffer_size=60000)
dataset = dataset.map(preprocess_triplets)

# Let's now split our dataset in train and validation.
train_dataset = dataset.take(round(image_count * 0.8))
val_dataset = dataset.skip(round(image_count * 0.8))

train_dataset = train_dataset.batch(32, drop_remainder=False)
train_dataset = train_dataset.prefetch(8)

val_dataset = val_dataset.batch(32, drop_remainder=False)
val_dataset = val_dataset.prefetch(8)

In [25]:
def visualize(anchor, positive, negative):
    """Visualize a few triplets from the supplied batches."""

    def show(ax, image):
        ax.imshow(image)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

    fig = plt.figure(figsize=(9, 9))

    axs = fig.subplots(3, 3)
    for i in range(3):
        show(axs[i, 0], anchor[i])
        show(axs[i, 1], positive[i])
        show(axs[i, 2], negative[i])

visualize(*list(train_dataset.take(1))[0])

In [26]:
# embedding_pred = tf.keras.models.load_model('/Users/davidschwartz/Documents/GitHub/IML-Projects/Task 3/save_embedding_model')
# embedding_pred.compile(optimizer=optimizers.Adam(0.0001))

In [27]:
base_cnn = resnet.ResNet50(
    weights="imagenet", input_shape=target_shape + (3,), include_top=False
)

flatten = layers.Flatten()(base_cnn.output)
dense1 = layers.Dense(512, activation="relu")(flatten)
dense1 = layers.BatchNormalization()(dense1)
dense2 = layers.Dense(256, activation="relu")(dense1)
dense2 = layers.BatchNormalization()(dense2)
output = layers.Dense(256)(dense2)

embedding = Model(base_cnn.input, output, name="Embedding")

trainable = False
for layer in base_cnn.layers:
    if layer.name == "conv5_block1_out":
        trainable = True
    layer.trainable = trainable

In [28]:
class DistanceLayer(layers.Layer):
    """
    This layer is responsible for computing the distance between the anchor
    embedding and the positive embedding, and the anchor embedding and the
    negative embedding.
    """

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, anchor, positive, negative):
        ap_distance = tf.reduce_sum(tf.square(anchor - positive), -1)
        an_distance = tf.reduce_sum(tf.square(anchor - negative), -1)
        return (ap_distance, an_distance)


anchor_input = layers.Input(name="anchor", shape=target_shape + (3,))
positive_input = layers.Input(name="positive", shape=target_shape + (3,))
negative_input = layers.Input(name="negative", shape=target_shape + (3,))

distances = DistanceLayer()(
    embedding(resnet.preprocess_input(anchor_input)),
    embedding(resnet.preprocess_input(positive_input)),
    embedding(resnet.preprocess_input(negative_input)),
)

siamese_network = Model(
    inputs=[anchor_input, positive_input, negative_input], outputs=distances
)

In [29]:
class SiameseModel(Model):
    """The Siamese Network model with a custom training and testing loops.

    Computes the triplet loss using the three embeddings produced by the
    Siamese Network.

    The triplet loss is defined as:
       L(A, P, N) = max(‖f(A) - f(P)‖² - ‖f(A) - f(N)‖² + margin, 0)
    """

    def __init__(self, siamese_network, margin=0.5):
        super(SiameseModel, self).__init__()
        self.siamese_network = siamese_network
        self.margin = margin
        self.loss_tracker = metrics.Mean(name="loss")

    def call(self, inputs):
        return self.siamese_network(inputs)

    def train_step(self, data):
        # GradientTape is a context manager that records every operation that
        # you do inside. We are using it here to compute the loss so we can get
        # the gradients and apply them using the optimizer specified in
        # `compile()`.
        with tf.GradientTape() as tape:
            loss = self._compute_loss(data)

        # Storing the gradients of the loss function with respect to the
        # weights/parameters.
        gradients = tape.gradient(loss, self.siamese_network.trainable_weights)

        # Applying the gradients on the model using the specified optimizer
        self.optimizer.apply_gradients(
            zip(gradients, self.siamese_network.trainable_weights)
        )

        # Let's update and return the training loss metric.
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def test_step(self, data):
        loss = self._compute_loss(data)

        # Let's update and return the loss metric.
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def _compute_loss(self, data):
        # The output of the network is a tuple containing the distances
        # between the anchor and the positive example, and the anchor and
        # the negative example.
        ap_distance, an_distance = self.siamese_network(data)

        # Computing the Triplet Loss by subtracting both distances and
        # making sure we don't get a negative value.
        loss = ap_distance - an_distance
        loss = tf.maximum(loss + self.margin, 0.0)
        return loss

    @property
    def metrics(self):
        # We need to list our metrics here so the `reset_states()` can be
        # called automatically.
        return [self.loss_tracker]

In [32]:
siamese_model = SiameseModel(siamese_network)
siamese_model.compile(optimizer=optimizers.Adam(0.0001))
checkpoint_path = "/Users/davidschwartz/Documents/GitHub/IML-Projects/Task 3/cp-{epoch:04d}.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, verbose=1, save_weights_only=True, save_freq='epoch')
siamese_model.load_weights(filepath='/Users/davidschwartz/Documents/GitHub/IML-Projects/Task 3/cp-0001.ckpt',)
siamese_model.save_weights(checkpoint_path.format(epoch=0))
siamese_model.fit(train_dataset, epochs=1, validation_data=val_dataset, callbacks=cp_callback)
#siamese_model.save('/content/drive/MyDrive/STUDIUM/Studium 2022 FS/Machine Learning/Projects/Project 3/save_siamese_model')
embedding.save('/Users/davidschwartz/Documents/GitHub/IML-Projects/Task 3/save_embedding_model')

NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for /Users/davidschwartz/Documents/GitHub/IML-Projects/Task 3/cp-0001.ckpt

In [ ]:
sample = next(iter(train_dataset))
visualize(*sample)

anchor, positive, negative = sample
anchor_embedding, positive_embedding, negative_embedding = (
    embedding(resnet.preprocess_input(anchor)),
    embedding(resnet.preprocess_input(positive)),
    embedding(resnet.preprocess_input(negative)),
)
ap_distance, an_distance = DistanceLayer()(
    embedding(resnet.preprocess_input(anchor_input)),
    embedding(resnet.preprocess_input(positive_input)),
    embedding(resnet.preprocess_input(negative_input)),
)

In [ ]:
print(ap_distance[0], an_distance[0])

In [ ]:
cosine_similarity = metrics.CosineSimilarity()
counter=0
for i in range(0, 32):
    pos=cosine_similarity(anchor_embedding[i], positive_embedding[i]).numpy()
    neg=cosine_similarity(anchor_embedding[i], negative_embedding[i]).numpy()
    print("%d : positive: %f, negative: %f" %(i,pos,neg))
    if pos>neg:
        counter = counter +1
print(counter/32)

In [ ]:
sample = next(iter(val_dataset))
visualize(*sample)

anchor, positive, negative = sample
anchor_embedding, positive_embedding, negative_embedding = (
    embedding(resnet.preprocess_input(anchor)),
    embedding(resnet.preprocess_input(positive)),
    embedding(resnet.preprocess_input(negative)),
)

In [ ]:
cosine_similarity = metrics.CosineSimilarity()

positive_similarity = cosine_similarity(anchor_embedding, positive_embedding)
print("Positive similarity:", positive_similarity.numpy())

negative_similarity = cosine_similarity(anchor_embedding, negative_embedding)
print("Negative similarity:", negative_similarity.numpy())

In [ ]:
test_anchor_indexes=test_triplets.iloc[0:59515:,0]
print(len(test_anchor_indexes))
print(test_anchor_indexes)
test_positives_indexes=test_triplets.iloc[0:59515,1]
test_negatives_indexes=test_triplets.iloc[0:59515,2]

In [ ]:
test_anchor, test_positive, test_negative = test_dataset
test_anchor_embedding, test_positive_embedding, test_negative_embedding = (
    embedding(resnet.preprocess_input(test_anchor)),
    embedding(resnet.preprocess_input(test_positive)),
    embedding(resnet.preprocess_input(test_negative)),
)

In [ ]:
test_dataset = tf.data.Dataset.zip((anchor_test_dataset, positive_test_dataset, negative_test_dataset))
test_dataset = test_dataset.shuffle(buffer_size=60000)
test_dataset = test_dataset.map(preprocess_triplets)